In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.pardir))
import sys
from argparse import ArgumentParser

import torch
from voltaml.compile import VoltaGPUCompiler
from voltaml.inference import gpu_performance
import torchvision
import segmentation_models_pytorch as smp

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pwd

/workspace/voltaML-1/demo


### Load Model 

In [2]:
# model = smp.DeepLabV3Plus(
#             encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#             encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#             in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#             classes=1,                      # model output channels (number of classes in your dataset)
#         )

model=torch.hub.load('pytorch/vision:v0.6.0', 'fcn_resnet101', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


## Set parameters for FP16

In [3]:
input_shape = (1,3,224,224)
precision = 'fp16'
compiled_model_dir = 'fcn_101.engine' ## Set Model dir
throughput_batch_size = 1

### Compile Model

In [4]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision
    # simplify=True
)

compiled_model = compiler.compile()

INFO:EngineBuilder:Network Description
Network Description
Network Description
INFO:EngineBuilder:Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
Input 'onnx::Conv_0' with shape (1, 3, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '1003' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '1003' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '1003' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Output '1022' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '1022' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
Output '1022' with shape (1, 21, 224, 224) and dtype DataType.FLOAT
INFO:EngineBuilder:Building fp16 Engine in /workspace/voltaML-1/demo/fcn_101.engine
Building fp16 Engine in /workspace/voltaML-1/demo/fcn_101.engine
Building fp16 Engine in /workspace/voltaML-1/demo/fcn_101.engine


[08/23/2022-13:05:27] [TRT] [I] [MemUsageChange] Init CUDA: CPU +313, GPU +0, now: CPU 1463, GPU 2545 (MiB)
[08/23/2022-13:05:27] [TRT] [I] [MemUsageSnapshot] Begin constructing builder kernel library: CPU 1463 MiB, GPU 2545 MiB
[08/23/2022-13:05:28] [TRT] [I] [MemUsageSnapshot] End constructing builder kernel library: CPU 1598 MiB, GPU 2579 MiB
[08/23/2022-13:05:28] [TRT] [W] parsers/onnx/onnx2trt_utils.cpp:364: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[08/23/2022-13:05:28] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +513, GPU +224, now: CPU 2319, GPU 2803 (MiB)
[08/23/2022-13:05:29] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +116, GPU +52, now: CPU 2435, GPU 2855 (MiB)
[08/23/2022-13:05:29] [TRT] [I] Local timing cache in use. Profiling results in this builder pass will not be stored.
[08/23/2022-13:05:48] [TRT] [I] Some tactics do not have sufficient workspace memory to run. Incr

INFO:EngineBuilder:Serializing engine to file: /workspace/voltaML-1/demo/fcn_101.engine
Serializing engine to file: /workspace/voltaML-1/demo/fcn_101.engine
Serializing engine to file: /workspace/voltaML-1/demo/fcn_101.engine


[08/23/2022-13:06:05] [TRT] [I] Total Host Persistent Memory: 281376
[08/23/2022-13:06:05] [TRT] [I] Total Device Persistent Memory: 108837888
[08/23/2022-13:06:05] [TRT] [I] Total Scratch Memory: 0
[08/23/2022-13:06:05] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 122 MiB, GPU 4415 MiB
[08/23/2022-13:06:05] [TRT] [I] [BlockAssignment] Algorithm ShiftNTopDown took 6.23747ms to assign 5 blocks to 113 nodes requiring 8467456 bytes.
[08/23/2022-13:06:05] [TRT] [I] Total Activation Memory: 8467456
[08/23/2022-13:06:05] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 3053, GPU 3187 (MiB)
[08/23/2022-13:06:05] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 3053, GPU 3195 (MiB)
[08/23/2022-13:06:05] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in building engine: CPU +103, GPU +104, now: CPU 103, GPU 104 (MiB)


In [5]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=1, is_yolo=False)

calculating latency...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 69.27it/s]


LogicError: cuMemcpyHtoD failed: invalid argument

### Set parameters for INT8

In [ ]:
input_shape = (1,3,224,224)
precision = 'int8'
compiled_model_dir = '' ## Compiled model directory
throughput_batch_size = 1
calib_input = '' ## Calib input images path
calib_cache = '' ## Cache name
calib_num_images=25000
calib_batch_size=8
calib_preprocessor='V2'

### Compile Model

In [ ]:
compiler = VoltaGPUCompiler(
    model=model,
    output_dir=compiled_model_dir,
    input_shape=input_shape,
    precision=precision,
    calib_input=calib_input,
    calib_cache=calib_cache,
    calib_num_images=calib_num_images,
    calib_batch_size=calib_batch_size,
    calib_preprocessor=calib_preprocessor
)

compiled_model = compiler.compile()

In [ ]:
gpu_performance(compiled_model_dir, model, input_shape=input_shape, throughput_batch_size=throughput_batch_size)